In [12]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [9]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '8a02c7990001d19b183ed9ec235b04e0', 'Date': 'Thu, 22 Aug 2024 02:28:17 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [10]:
import json
data = json.load(open("reviews.json"))
data['reviews'] 

[{'professor': 'John Doe',
  'review': 'Great professor, really knows how to explain complex concepts in a simple way.',
  'subject': 'Mathematics',
  'stars': 5},
 {'professor': 'Jane Smith',
  'review': 'Engaging lectures and very approachable during office hours.',
  'subject': 'Electrical Engineering',
  'stars': 4},
 {'professor': 'Michael Johnson',
  'review': 'Good professor but sometimes moves too quickly through the material.',
  'subject': 'Mathematics',
  'stars': 4},
 {'professor': 'Susan Lee',
  'review': 'Excellent at making difficult physics topics understandable.',
  'subject': 'Physics',
  'stars': 5},
 {'professor': 'David Brown',
  'review': 'Very knowledgeable but a bit strict with grading.',
  'subject': 'Computer Science',
  'stars': 3},
 {'professor': 'Emily Davis',
  'review': 'Amazing professor! Makes lectures interesting and fun.',
  'subject': 'Chemistry',
  'stars': 5},
 {'professor': 'William Clark',
  'review': 'Good lectures, but can be hard to get in tou

In [28]:
import time
import openai

processed_data = []
client = OpenAI()

for i, review in enumerate(data['reviews']):
    try:
        print(f"Processing review {i+1}/{len(data['reviews'])}")
        response = client.embeddings.create(
            input=review['review'],
            model="text-embedding-ada-002",
        )
        print(f"Received response for review {i+1}")

        embedding = response.data[0].embedding
        processed_data.append({
            "values": embedding,
            "id": review["professor"],
            "metadata": {
                "review": review["review"],
                "subject": review["subject"],
                "stars": review["stars"],
            }
        })
    except openai.error.OpenAIError as e:
        print(f"Error processing review {i+1}: {e}")
        continue  # Skip to the next review




Processing review 1/20
Received response for review 1
Processing review 2/20
Received response for review 2
Processing review 3/20
Received response for review 3
Processing review 4/20
Received response for review 4
Processing review 5/20
Received response for review 5
Processing review 6/20
Received response for review 6
Processing review 7/20
Received response for review 7
Processing review 8/20
Received response for review 8
Processing review 9/20
Received response for review 9
Processing review 10/20
Received response for review 10
Processing review 11/20
Received response for review 11
Processing review 12/20
Received response for review 12
Processing review 13/20
Received response for review 13
Processing review 14/20
Received response for review 14
Processing review 15/20
Received response for review 15
Processing review 16/20
Received response for review 16
Processing review 17/20
Received response for review 17
Processing review 18/20
Received response for review 18
Processing

In [29]:
processed_data[0]

{'values': [-0.0035123676,
  0.013546356,
  0.0113326525,
  -0.02905486,
  0.0027042401,
  0.007339182,
  -0.009508863,
  -0.0026869455,
  -0.021432675,
  -0.020275513,
  -0.0016901501,
  0.0032387993,
  -0.0038142365,
  0.011571633,
  0.011338942,
  -0.004169561,
  0.03728078,
  0.0089743035,
  0.011835767,
  -0.018225322,
  -0.024853854,
  0.0048298985,
  -0.014099782,
  -0.014615474,
  -0.009219572,
  0.023508022,
  0.02252695,
  -0.00011791745,
  -0.0010447486,
  -0.009565463,
  0.034035694,
  -0.004594064,
  -0.032551505,
  -0.029331572,
  0.0052135233,
  0.0012585722,
  -0.0101126,
  0.011118829,
  -0.006968135,
  -0.011804323,
  0.017445493,
  -0.005386469,
  0.015910994,
  -0.0063360976,
  -0.018011497,
  -0.0033771556,
  0.005493381,
  -0.045079056,
  -0.014929921,
  0.009936511,
  0.007534139,
  0.027595827,
  -0.0033897336,
  -0.020866672,
  -0.031243406,
  -0.02633804,
  -0.0040374934,
  0.019772397,
  0.01334511,
  -0.0318723,
  -0.010030844,
  -0.008156743,
  -0.024916742

In [30]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 20}

In [31]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}